# Comparing MASH analysis with simple multivariate analysis

Previously we showed that even though univariate analysis with degenerated MASH model gives identical results to SuSiE as expected (with non-decreasing ELBO), in multivariate calculations the ELBO is not always non-decreasing. To investigate the issue we will 1) further simplify the problem and 2) isolate the problem to posterior calculation versus ELBO calculations and check which part is problematic. The best way to achieve both is to implement a simple Bayesian multivariate regression model with prior $b \sim MVN(0, U)$ where $U$ is known, instead of using MASH prior for $b$.

This feature is now implemented in the [`BayesianMultivariateRegression` class](https://github.com/gaow/mmbr/blob/master/R/bayesian_multivariate_regression.R) with an interface added to the main function such that the code will be triggered [when prior variance is a matrix](https://github.com/gaow/mmbr/blob/dccc26cfbba0cfd3e6f20b37e06d4db6305b834f/R/main.R#L110). 

With this feature (and with Yuxin's sharp eyes!!) we were able to identify an issue caused by inconsistent interface between `mmbr::susie` and `mmbr::MashInitializer` in handling residual variances. After patching the issue (interface fix still needs to be finalized) we are able to get consistent result between simple Bayesian multivariate regression and MASH; and MASH ELBO now increases.

## Test the calculation agree with univariate code

In [1]:
library(mmbr)
set.seed(2)
L = 5

Loading required package: mashr
Loading required package: ashr


In [2]:
dat = mmbr_sim1(r=1)

Run the simulated univariate data with SuSiE,

In [3]:
res = susieR::susie(dat$X,dat$y,L=L,scaled_prior_variance=0.2,estimate_residual_variance=F,estimate_prior_variance=F)
res$elbo

[1] -429.4050 -409.5493 -407.7077 -407.7058 -407.7058

Now run it with multivariate simple prior implementation in `mmbr`,

In [4]:
res = mmbr::msusie(dat$X,dat$y,L=L,prior_variance=0.2*var(dat$y),compute_objective=T,estimate_residual_variance=F,estimate_prior_variance=F)
res$elbo

[1] -429.4050 -409.5493 -407.7077 -407.7058 -407.7058

So it is confirmed that this implementation produces identical results as SuSiE runs.

## Test multivariate analysis

In [5]:
set.seed(2)
dat = mmbr_sim1(r=5)

In [6]:
res = mmbr::msusie(dat$X,dat$y,L=L,prior_variance=dat$V,compute_objective=T,estimate_residual_variance=F,estimate_prior_variance=F)
res$elbo

[1] -2184.076 -2173.199 -2169.551 -2167.744 -2167.267 -2167.087 -2166.514
 [8] -2166.013 -2164.894 -2163.635 -2163.574 -2163.573 -2163.573

Here the ELBO is non-decreasing, as expected.

## Compare with MASH based model

In [7]:
m_init = mmbr:::MashInitializer$new(list(dat$V), 1, prior_weight=1, null_weight=0,alpha=0)

In [8]:
res = mmbr::msusie(dat$X,dat$y,L=L,prior_variance=m_init,compute_objective=T,estimate_residual_variance=F,estimate_prior_variance=F)
res$elbo

[1] -2184.076 -2173.199 -2169.551 -2167.744 -2167.267 -2167.087 -2166.514
 [8] -2166.013 -2164.894 -2163.635 -2163.574 -2163.573 -2163.573

The result agree with above. Now we use a different prior choice -- a diagonal prior covariance. We analyze with simple Bayesian multivariate regression,

In [9]:
res = mmbr::msusie(dat$X,dat$y,L=L,prior_variance=0.2*diag(ncol(dat$y)),compute_objective=T,estimate_residual_variance=F,estimate_prior_variance=F)
res$elbo

[1] -2174.053 -2166.188 -2161.716 -2158.706 -2156.964 -2156.886 -2156.884
[8] -2156.884

and with MASH based regression,

In [10]:
m_init = mmbr:::MashInitializer$new(list(0.2*diag(ncol(dat$y))), 1, prior_weight=1, null_weight=0,alpha=0)

In [11]:
res = mmbr::msusie(dat$X,dat$y,L=L,prior_variance=m_init,compute_objective=T,estimate_residual_variance=F,estimate_prior_variance=F)
res$elbo

[1] -2174.053 -2166.188 -2161.716 -2158.706 -2156.964 -2156.886 -2156.884
[8] -2156.884

So we are comfortable at this point that ELBO for multivariate analysis is done correctly.